<a href="https://colab.research.google.com/github/Fudorel/tugas4_kecerdasanbuatan/blob/main/Dedupe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install sparse_dot_topn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.7/266.7 kB 9.2 MB/s eta 0:00:00


In [36]:
from IPython import get_ipython
from IPython.display import display

In [38]:
from scipy.sparse import csr_matrix
import sparse_dot_topn as ct
import sparse_dot_topn
from sklearn.feature_extraction.text import TfidfVectorizer

In [45]:
from sparse_dot_topn import awesome_cossim_topn
import numpy as np

In [43]:
pip install --upgrade sparse_dot_topn

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!pip install pandas

In [10]:
import pandas as pd

In [11]:
df = pd.read_csv('/content/drive/MyDrive/Farit_Fudo/Seattle_Hotels_Duplicates.csv', encoding="latin-1")

In [13]:
df.head(3)

,name,address
0,Hilton Garden Inn Seattle Downtown,"1821 Boren Avenue, Seattle Washington 98101 USA"
1,Sheraton Grand Seattle,"1400 6th Avenue, Seattle, Washington 98101 USA"
2,Crowne Plaza Seattle Downtown,"1113 6th Ave, Seattle, WA 98101"


In [12]:
df.name.value_counts()

,count
name,
Hilton Garden Inn Seattle Downtown,2
Ace Hotel Seattle,2
citizenM Seattle South Lake Union hotel,2
Renaissance Seattle Hotel,2
Quality Inn & Suites Seattle Center,2
...,...
Mildred's Bed and Breakfast,1
Hampton Inn Seattle/Southcenter,1
The Halcyon Suite Du Jour,1


In [14]:
df.loc[df['name'] == 'Roy Street Commons']

,name,address
82,Roy Street Commons,"621 12th Ave E, Seattle, WA 98102"
90,Roy Street Commons,"621 12th Avenue East, Seattle, Washington 98102"


In [16]:
# df['name_address'] = df['name'] + ' ' + df['address']
# name_address = df['name_address']
# The 'char' argument was incorrectly passed as a positional argument.
# To perform character-based analysis, use the 'analyzer' parameter.
vectorizer = TfidfVectorizer(analyzer="char", ngram_range=(1, 4), sublinear_tf=True)
tf_idf_matrix = vectorizer.fit_transform(name_address)

In [17]:
tf_idf_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 32557 stored elements and shape (168, 4957)>

In [46]:
matches = awesome_cossim_topn(tf_idf_matrix, tf_idf_matrix.transpose(), ntop=5, lower_bound=0.8)

<ipython-input-46-70da71f064e4>:1: DeprecationWarning: `awesome_cossim_topn` function will be removed and (partially) replaced with `sp_matmul_topn`. See the migration guide at 'https://github.com/ing-bank/sparse_dot_topn#readme'.
  matches = awesome_cossim_topn(tf_idf_matrix, tf_idf_matrix.transpose(), ntop=5, lower_bound=0.8)


In [19]:
import numpy as np

In [24]:
!pip install sparse-dot-topn

In [47]:
def get_matches_df(sparse_matrix, name_vector, top=None): # Changed default top to None as in original notebook
    non_zeros = sparse_matrix.nonzero()

    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]

    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size

    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)

    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]

    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similarity': similairity})

matches_df = get_matches_df(matches, name_address)
#Displaying the head of the matches_df as the original notebook did,
display(matches_df[matches_df['similarity'] < 0.99999].sort_values(by=['similarity'], ascending=False).head(30))

,left_side,right_side,similarity
195,Pike's Place Lux Suites by Barsala 2rd Ave and...,Pike's Place Lux Suites by Barsala 2nd Ave and...,0.932024
193,Pike's Place Lux Suites by Barsala 2nd Ave and...,Pike's Place Lux Suites by Barsala 2rd Ave and...,0.932024
1,Hilton Garden Inn Seattle Downtown 1821 Boren ...,Hilton Garden Inn Seattle Downtown 1821 Boren ...,0.911902
12,Hilton Garden Inn Seattle Downtown 1821 Boren ...,Hilton Garden Inn Seattle Downtown 1821 Boren ...,0.911902
130,Quality Inn & Suites Seattle Center 618 John S...,Quality Inn & Suites Seattle Center 618 John S...,0.889089
139,Quality Inn & Suites Seattle Center 618 John S...,Quality Inn & Suites Seattle Center 618 John S...,0.889089
128,"Renaissance Seattle Hotel 515 Madison St, Seat...","Renaissance Seattle Hotel 515 Madison Street, ...",0.874515
133,"Renaissance Seattle Hotel 515 Madison Street, ...","Renaissance Seattle Hotel 515 Madison St, Seat...",0.874515
79,Home2 Suites by Hilton Seattle Airport 380 Upl...,Home2 Suites by Hilton Seattle Airport 380 Upl...,0.873906
81,Home2 Suites by Hilton Seattle Airport 380 Upl...,Home2 Suites by Hilton Seattle Airport 380 Upl...,0.873906


In [48]:
matches

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 198 stored elements and shape (168, 168)>

In [50]:
def get_matches_df(sparse_matrix, name_vector, top=None):
    non_zeros = sparse_matrix.nonzero()

    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]

    # The number of non-zero matches is the size of the non_zeros arrays
    nr_matches = sparsecols.size

    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)

    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]

    df_matches = pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similarity': similairity})

    # If top is specified, return only the top N matches
    if top is not None:
        # It's generally better to sort by similarity and take the top N if
        # you want the best matches, rather than just the first N encountered
        # in the sparse matrix's internal representation.
        # However, to match the apparent intent of using 'top' as a limit
        # on the number of rows returned, we can simply slice the DataFrame.
        # If you need the *best* matches, sort by 'similarity' first.
        return df_matches.head(top)
    else:
        return df_matches

# Call the function with the desired parameters
matches_df = get_matches_df(matches, name_address, top=840) # Or remove top if you want all matches

In [51]:
matches_df[matches_df['similarity'] < 0.99999].sort_values(by=['similarity'], ascending=False).head(30)

,left_side,right_side,similarity
195,Pike's Place Lux Suites by Barsala 2rd Ave and...,Pike's Place Lux Suites by Barsala 2nd Ave and...,0.932024
193,Pike's Place Lux Suites by Barsala 2nd Ave and...,Pike's Place Lux Suites by Barsala 2rd Ave and...,0.932024
1,Hilton Garden Inn Seattle Downtown 1821 Boren ...,Hilton Garden Inn Seattle Downtown 1821 Boren ...,0.911902
12,Hilton Garden Inn Seattle Downtown 1821 Boren ...,Hilton Garden Inn Seattle Downtown 1821 Boren ...,0.911902
130,Quality Inn & Suites Seattle Center 618 John S...,Quality Inn & Suites Seattle Center 618 John S...,0.889089
139,Quality Inn & Suites Seattle Center 618 John S...,Quality Inn & Suites Seattle Center 618 John S...,0.889089
128,"Renaissance Seattle Hotel 515 Madison St, Seat...","Renaissance Seattle Hotel 515 Madison Street, ...",0.874515
133,"Renaissance Seattle Hotel 515 Madison Street, ...","Renaissance Seattle Hotel 515 Madison St, Seat...",0.874515
79,Home2 Suites by Hilton Seattle Airport 380 Upl...,Home2 Suites by Hilton Seattle Airport 380 Upl...,0.873906
81,Home2 Suites by Hilton Seattle Airport 380 Upl...,Home2 Suites by Hilton Seattle Airport 380 Upl...,0.873906


In [52]:
matches_df[matches_df['similarity'] < 0.50].right_side.nunique()

0